In [1]:
import os
git_dir = os.path.dirname(os.getcwd())
project_dir = os.path.join(git_dir,'zavmo')
os.chdir(project_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
from helpers.chat import get_openai_completion,make_function_call,get_prompt, make_structured_call
# from helpers.functions import OpenAISchema
from IPython.display import Markdown
import json
import anthropic

In [4]:
get_openai_completion([{"role":"user","content":"Hi"}])

'Hello! How can I assist you today?'

In [8]:
from enum import Enum
from typing import Union, List, Optional
from pydantic import BaseModel, Field
import openai
from openai import OpenAI

class GetFirstName(BaseModel):
    "Return the first name of the user"
    first_name: str = Field(
        description="The first name of the user, often used to address them directly, e.g., 'Diana' or 'John'."
    )

class GetLastName(BaseModel):
    "Return the user's family name or surname"
    last_name: str = Field(
        description="The last name (surname) of the user, typically following the first name, e.g., 'Smith' or 'Haddad'."
    )

class GetCurrentRole(BaseModel):
    current_role: str = Field(
        description="The specific job position the person is currently in."
    )

class GetLearningInterests(BaseModel):
    """Topics or subjects related information shared"""
    learning_interests: List[str] = Field(
        description="A list of specific topic or subject the user is interested in learning."
    )

class Confirmed(str,Enum):
    yes = "Yes"
    no = "No"

class GetConfirmation(BaseModel):
    """Confirmation from user whether the shared info was correct"""
    answer: Optional[Confirmed]

In [10]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [9]:
def make_tool_calls(messages, tools, model="gpt-4o-mini", **kwargs):
    """Completes the chat given a list of messages.
            
    Args:
        messages (list): list of messages to complete the chat with
        **kwargs: additional arguments to pass to the OpenAI API, such as
            temperature, max_tokens, etc.
    Returns:
        str: the chat completion
    """
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        parallel_tool_calls=True,
        **kwargs,
    )        
    return response.choices[0].message.tool_calls

In [7]:
class GetLearningGoal(BaseModel):
    """The primary learning goal of the user."""
    learning_goal: str = Field(
        description="The main learning objective that the user aims to achieve, e.g., 'Improve my coding skills'."
    )

# Class for Importance of the Goal
class GetGoalImportance(BaseModel):
    """The reason why the learning goal is important to the user."""
    goal_importance: str = Field(
        description="The user's explanation of why this learning goal matters to them."
    )

# Class for Current Knowledge Rating
class GetKnowledgeRating(BaseModel):
    """User's self-assessment of their current knowledge level."""
    knowledge_rating: int = Field(
        description="A self-rated score from 1 to 5 reflecting the user's current knowledge of the subject."
    )

In [17]:
sys_mssg = """You are Zavmo, an AI educational companion. 

Your primary goal is to frame questions around set of required fields that will be shared and you are expected to gather responses in order to create the user's learning profile. 

Focus strictly on asking only from the required fields.

### Progress Tracking:

- **Required Fields**: Only ask questions for fields that are mentioned under required fields section.
- **Completed Fields**: Do not ask questions related to any field addressed completed as they fall under completed fields.

### Interaction Tips:

- Maintain a friendly, conversational tone throughout all interactions.
- Start by introducing yourself and explaining that you'll ask a few questions to personalize their learning experience.
- Ask each question individually in a friendly and warm tone. For example: “Hi! What's your first name?”
- Award 10 XP for each answer to celebrate progress.
- Personalize with their name, emojis, and friendly phrases.
- If the user hesitates, gently encourage them to share.

### Guidelines:

- **Avoid Repetition**: If the user answers a question before being asked, mark it as completed and skip it. Regularly refer to `Completed Fields` to avoid redundancy.
- **Completion**: 
  - If the required fields still exist then push the user to answer sharing the importance of having them in the user's learning journey.
  - Once all remaining fields are answered and there are no further questions pending:
    - Summarize the completed information and confirm with the user. See sample confirmation below.
    - Politely close the conversation without asking additional questions.

### Sample Confirmation Message:

Once all fields are completed, you can confirm by saying:
"Awesome! 🎉 Your profile is now complete. Here's what we've got so far from the present stage:
[Field Name 1]: [Value 1]
[Field Name 2]: [Value 2]
.
.
[Field Name n]: [Value n]

Ask for confirmation (Yes/No) sharing Noted Information.
"""

In [ ]:
confirmation_instruction = """"Ask for confirmation (Yes/No) sharing Noted Information

### Sample Confirmation Message:

Once all fields are completed, you can confirm by saying:
"Awesome! 🎉 Your profile is now complete. Here's what we've got so far from the present stage:
First Name: "Alice",
Last Name: "Smith",
Current Role": "Product Manager",
Learning Interests": ["UX Design", "Agile Methodologies", "Data Analysis"]"""

In [979]:
# Extractor Bot Logic
class ExtractorBot:
    def __init__(self, required_fields,tools):
        self.profile_data = {}
        self.required_fields = required_fields
        self.completed_fields = []
        self.tools = tools

    def extract_data(self, messages):
        # Prepare and send messages for tool calls
        tool_sys_msg =extra.format(
            required_fields=", ".join(self.required_fields) if self.required_fields else "No pending fields to extract"
        )
        tool_messages = [{"role": "system", "content": tool_sys_msg}] + messages[1:]
        tool_calls = make_tool_calls(tool_messages, self.tools)

        if tool_calls:
            for tool_response in tool_calls:
                arg = tool_response.function.arguments
                self.profile_data.update(json.loads(arg))

            # Update fields status
            self.completed_fields = list(set(self.profile_data))
            self.required_fields = [field for field in self.required_fields if field not in self.completed_fields]

    def are_all_fields_collected(self):
        return not self.required_fields

    def get_confirmation(self, messages):
        answer = make_structured_call(GetConfirmation, messages).answer
        return 'Yes' if answer and answer.value == 'Yes' else 'No'


# Guiding Bot Logic
class ProfileCollector:
    def __init__(self, extractor_bot):
        self.extractor_bot = extractor_bot
        self.history = []
        self.confirmed = "No"

    def system_message(self):
        prompt = get_prompt("system.md", "assets/prompts/profile")
        if self.extractor_bot.are_all_fields_collected():
            prompt += f"Information Noted:\n\n```json\n{json.dumps(self.extractor_bot.profile_data, indent=4)}\n```"
        return {"role": "system", "content": prompt}

    def user_message(self, input=""):
        # Generate user message with current field statuses
        completed = ", ".join(self.extractor_bot.completed_fields) or "Not completed any field yet."
        required = ", ".join(self.extractor_bot.required_fields) or "Received information for all remaining fields."
        content = f"Current Stage Information\n\nCompleted Fields:\n{completed}\n\nRequired Fields:\n{required}\n\nUser response:\n{input}"
        self.user_mssg = {"role": "user", "content": content}

    def ask_question(self):
        # Check if all fields are collected and ask for confirmation if needed
        try: 
            self.user_mssg
        except:
            self.user_message()

        if self.extractor_bot.are_all_fields_collected():
            user_mssg = self.user_mssg 
            user_mssg['content'] = user_mssg['content'] + "\n\nIf updates on any field is required and user will mention corresponding to what is asked, or if user says No. Take it as a No"
            messages = [self.system_message()] + self.history + [user_mssg]
            confirmation = self.extractor_bot.get_confirmation(messages)
            
            if confirmation == "Yes":
                self.confirmed = "Yes"
                return self.summarize_profile()

        # Collect remaining fields or reattempt extraction if confirmation is "No"
        if not self.extractor_bot.are_all_fields_collected() or self.confirmed != "Yes":
            tool_messages = self.history + [self.user_mssg]
            self.extractor_bot.extract_data(tool_messages)

            # Generate and return assistant message
            messages = [self.system_message()] + self.history + [self.user_mssg]
            assistant_message = get_openai_completion(messages)
            messages.append({"role": "assistant", "content": assistant_message})
            self.history = messages[1:]  # Update history
            return assistant_message

    def summarize_profile(self):
        return f"Great! Your profile is complete:\n\n```json\n{json.dumps(self.extractor_bot.profile_data, indent=4)}\n```"


In [13]:
required_fields_profile  = ["first_name", "last_name", "current_role", "learning_interests"]
profile_tools            = [GetFirstName, GetLastName, GetCurrentRole, GetLearningInterests,GetConfirmation]

required_fields_discover = ["learning_goal", "goal_importance", "knowledge_rating"]
discover_tools           = [GetLearningGoal, GetGoalImportance, GetKnowledgeRating]

In [14]:
tools = [openai.pydantic_function_tool(i) for i in profile_tools]

In [996]:
extractor_bot = ExtractorBot()
pc            = ProfileCollector(extractor_bot)

In [1007]:
Markdown(pc.ask_question())

Great! Your profile is complete:

```json
{
    "first_name": "Mumtaz",
    "last_name": "",
    "current_role": "",
    "learning_interests": []
}
```

In [1006]:
pc.user_message("Yes")

In [982]:
pc.user_message("Mumtaz Rahmani, current role is Junior software engineer, learning interests are invloved around LLMs and stats")

In [978]:
pc.extractor_bot.profile_data

{'first_name': 'Mumtaz',
 'last_name': 'R',
 'current_role': 'Junior Software Engineer',
 'learning_interests': ['LLMs', 'Stats']}

In [769]:
messages = pc.history

tool_sys_mssg = get_prompt('extractor.md','assets/prompts/profile').format(required_fields=", ".join(pc.extractor_bot.required_fields) if pc.extractor_bot.required_fields else "No pendig fields to extract")
tool_messages = [{"role":"system","content":tool_sys_mssg}]
tool_messages.extend(messages) 

In [619]:
messages.append({"role":"user","content":"Yes"})

In [626]:
answer = make_structured_call(GetConfirmation,messages).answer
if answer:
    print(answer.value)

Yes


## Internally sending tool response to model 

In [ ]:
profile_data={}

In [613]:
message = get_openai_completion(messages, tools=tools, tool_choice="auto", parallel_tool_calls=True)

tool_calls = message.tool_calls
if tool_calls:
    print(f"Tools called: {[tool.function.name for tool in tool_calls]}")
    function_call_messages=[message]
    for tool in tool_calls:
        arguments = tool.function.arguments
        function_call_messages.extend([{"role": "tool", "content": json.dumps(arguments), "tool_call_id": tool.id}])
        profile_data.update(json.loads(arguments))
    messages.extend(function_call_messages)
    while message.content==None:
        message=get_openai_completion(messages, tools=tools, tool_choice="auto", parallel_tool_calls=True)
        print("Running in while loop")

assistant_response = message.content

In [614]:
Markdown(assistant_response)

Hi there! I'm Zavmo, your AI educational companion. 🙌 I'm excited to help personalize your learning experience, and I just need to ask you a few simple questions. Let's get started!

What's your first name? 😊

In [615]:
messages.append({"role":"assistant","content":assistant_response})

In [476]:
profile_data

{}

In [ ]:
def get_claude_response(sys_mssg,messages,model="claude-3-opus-20240229"):
    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    message = client.messages.create(
        model=model,
        max_tokens=1024,
        system=sys_mssg,
        messages=messages
    )
    return message.content[0].text

claude_response = get_claude_response(sys_mssg,messages[1:])
claude_response